# AKSSF Hydrography
## Construct Synthetic Stream Network and Catchments using TauDEM tools and Timm's 10m composite DEM clipped to Study Area

### Set environments, variables, and import modules

In [ ]:
# path to TauDEM toolbox on local machine
TauLoc = r"C:\Program Files\TauDEM\TauDEM5Arc\TauDEM Tools.tbx"
# 10m Composite DEM Clipped to Study Area
DEM = ''
# Input feature class describing study area and processing regions that can be iterated over
study_area = ''
# Folder to store outputs
datafolder = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\data\\Outputs"


print("test")

### DEM Processing with TauDEM
Define Hydroprocessing function


In [ ]:
def hydroprocess(**kwargs):
    # Import packages
    import os
    import arcpy
    import datetime
    import time

    """
    Description: Creates synthetic stream network and catchments from input DEM
    :param Input_Number_of_Processes: 10
    :param Input_DEM: demclip #Dem clipped to study area
    :param Check_for_edge_contamination:False #Dem clipped to watershed boundary so keep false
    :param Outputs: region_out # name of processing region
    :param _Name_catchments_shp: catchname
    :param Output_Stream_Reach_file: streamname
    :return:
    """

    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True

    # Import TauDEM tools
    arcpy.ImportToolbox(TauLoc)

    # Use two thirds of cores on processes that can be split.
    arcpy.env.parallelProcessingFactor = "66%"

    # Parse key word argument inputs
    outfol = kwargs['data_folder']
    Input_DEM = kwargs['demclip']
    region = kwargs['processing_region']
    Input_Number_of_Processes = kwargs['10']
    Check_for_edge_contamination = kwargs['False']


    # Process: Pit Remove
    DEM_fillname = region + "_fel.tif"
    DEM_fill = os.path.join(outfol, DEM_fillname)

    if not os.path.exists(DEM_fill):
        # Start timing function
        iteration_start = time.time()
        arcpy.PitRemove(Input_Elevation_Grid=Input_DEM, Fill_Considering_only_4_way_neighbors=False,
                         Input_Depression_Mask_Grid="", Input_Number_of_Processes=Input_Number_of_Processes,
                         Output_Pit_Removed_Elevation_Grid=DEM_fill)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Pits filled at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print("Pit Filled Elevation Grid already exists ")
        print('----------')
    DEM_fill = arcpy.Raster(DEM_fill)

    # Process: D8 Flow Directions (D8 Flow Directions) ()
    _Name_p_tifname = region + "_p.tif"
    _Name_sd8_tifname = region + "_sd8.tif"
    _Name_p_tif = os.path.join(outfol, _Name_p_tifname)
    _Name_sd8_tif = os.path.join(outfol,_Name_sd8_tifname)
    if not os.path.exists(_Name_p_tif):
        # Start timing function
        iteration_start = time.time()
        arcpy.D8FlowDir(Input_Pit_Filled_Elevation_Grid=DEM_fill,
                         Input_Number_of_Processes=Input_Number_of_Processes,
                         Output_D8_Flow_Direction_Grid=_Name_p_tif, Output_D8_Slope_Grid=_Name_sd8_tif)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Flow Direction calculated at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print("Flow Direction Grid already exists")
        print('----------')
    _Name_p_tif = arcpy.Raster(_Name_p_tif)
    _Name_sd8_tif = arcpy.Raster(_Name_sd8_tif)

    # Process: D8 Contributing Area (D8 Contributing Area) ()
    _Name_ad8_tifname = region + "_ad8.tif"
    _Name_ad8_tif = os.path.join(outfol, _Name_ad8_tifname)
    if not os.path.exists(_Name_ad8_tif):
        # Start timing function
        iteration_start = time.time()
        arcpy.D8ContributingArea(Input_D8_Flow_Direction_Grid=_Name_p_tif,
                                  Input_Outlets="", Input_Weight_Grid="",
                                  Check_for_edge_contamination=Check_for_edge_contamination,
                                  Input_Number_of_Processes=Input_Number_of_Processes,
                                  Output_D8_Contributing_Area_Grid=_Name_ad8_tif)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'D8 Contributing area calculated at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print ('D8 Contributing Area Grid Already Exists')
        print('----------')
    _Name_ad8_tif = arcpy.Raster(_Name_ad8_tif)

    # Process: Peuker Douglas Stream Definition (Peuker Douglas Stream Definition) ()
    _Name_ss_tifname = region + "_ss.tif"
    _Name_ssa_tifname = region + "_ssa.tif"
    _Name_src_tifname = region + "_src.tif"
    _Name_drp_txtname = region + "_drp.txt"
    _Name_ss_tif = os.path.join(outfol, _Name_ss_tifname)
    _Name_ssa_tif = os.path.join(outfol, _Name_ssa_tifname)
    _Name_src_tif = os.path.join(outfol, _Name_src_tifname)
    _Name_drp_txt = os.path.join(outfol, _Name_drp_txtname)

    if not os.path.exists(_Name_src_tif):
        # Start timing function
        iteration_start = time.time()
        arcpy.PeukerDouglasStreamDef(Input_Elevation_Grid=DEM_fill,
                                      Input_D8_Flow_Direction_Grid=_Name_p_tif,
                                      Weight_Center=0.4, Weight_Side=0.1, Weight_Diagonal=0.05,
                                      Accumulation_Threshold=75,
                                      Check_for_Edge_Contamination=Check_for_edge_contamination,
                                      Input_Outlets="", Input_Mask_Grid="",
                                      Input_D8_Contributing_Area_for_Drop_Analysis=_Name_ad8_tif,
                                      Input_Number_of_Processes=Input_Number_of_Processes,
                                      Output_Stream_Source_Grid=_Name_ss_tif,
                                      Output_Accumulated_Stream_Source_Grid=_Name_ssa_tif,
                                      Output_Stream_Raster_Grid=_Name_src_tif,
                                      Output_Drop_Analysis_Table=_Name_drp_txt,
                                      Use_the_range_below_to_automatically_select_threshold_by_drop_analysis=True,
                                      Minimum_Threshold_Value=20, Maximum_Threshold_Value=500,
                                      Number_of_Threshold_Values=15,
                                      Use_Logarithmic_spacing_for_threshold_values=True)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Stream Definition completed at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print('Streams Already Defined')
        print('----------')
    _Name_ss_tif = arcpy.Raster(_Name_ss_tif)
    _Name_ssa_tif = arcpy.Raster(_Name_ssa_tif)
    _Name_src_tif = arcpy.Raster(_Name_src_tif)

    # Process: Stream Reach And Watershed (Stream Reach And Watershed) ()
    Output_Stream_Reach_file = region + '_stream_reach.shp'
    Output_Stream_Order_Gridname = region + "_ord.tif"
    Output_Network_Connectivity_Treename = region + "_tree.txt"
    Output_Network_Coordinatesname = region + "_coord.txt"
    Output_Watershed_Gridname = region + "_w.tif"
    Output_Stream_Order_Grid = os.path.join(outfol, Output_Stream_Order_Gridname)
    Output_Network_Connectivity_Tree =  os.path.join(outfol, Output_Network_Connectivity_Treename)
    Output_Network_Coordinates = os.path.join(outfol, Output_Network_Coordinatesname)
    Output_Watershed_Grid = os.path.join(outfol, Output_Watershed_Gridname)

    if not os.path.exists(Output_Stream_Order_Grid):
        # Start timing function
        iteration_start = time.time()
        arcpy.StreamReachAndWatershed(Input_Pit_Filled_Elevation_Grid=DEM_fill,
                                       Input_D8_Flow_Direction_Grid=_Name_p_tif,
                                       Input_D8_Drainage_Area=_Name_ad8_tif,
                                       Input_Stream_Raster_Grid=_Name_src_tif,
                                       Input_Outlets="", Delineate_Single_Watershed=False,
                                       Input_Number_of_Processes=Input_Number_of_Processes,
                                       Output_Stream_Order_Grid=Output_Stream_Order_Grid,
                                       Output_Network_Connectivity_Tree=Output_Network_Connectivity_Tree,
                                       Output_Network_Coordinates=Output_Network_Coordinates,
                                       Output_Stream_Reach_file=Output_Stream_Reach_file,
                                       Output_Watershed_Grid=Output_Watershed_Grid)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Stream Reach and Watershed completed at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print ('Stream Reach and Watershed already identified')
        print('----------')
    Output_Stream_Order_Grid = arcpy.Raster(Output_Stream_Order_Grid)
    Output_Watershed_Grid = arcpy.Raster(Output_Watershed_Grid)

    # Process: Watershed Grid To Shapefile
    _Name_catchments_shpname = region + "_watersheds.shp"
    _Name_catchments_shp = os.path.join(outfol, _Name_catchments_shpname)
    if not os.path.exists(_Name_catchments_shp):
        arcpy.WaterShedGridToShapefile(Input_Watershed_Grid=Output_Watershed_Grid,
                                       Output_Watershed_Shapefile=_Name_catchments_shp)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Watershed Exported at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print("Watershed Grid already exported to shapefile")
        print('----------')

print ('ALL PROCESSES COMPLETE')

In [ ]:
# Create key word arguments
import arcpy

#processing_regions = ['Egegik']
#clause = 'Processing_Region' = processing_region
#roi = arcpy.MakeFeatureLayer_management(study_area, clause)
#demclip = arcpy.Clip_management(DEM,)
# hydroprocess_kwargs = {'data_folder': data_folder,
#                        'demclip': demclip,
#                        'processing_region': region,
#                        '10':10,
#                        'False':False}

